In [ ]:
from typing import Tuple, List

import torch
from torch import nn, Tensor
import torchaudio

In [ ]:
import sys

LIB_PATH = '/content/drive/MyDrive/GSC/GSC_helper'
sys.path.append(LIB_PATH)

from SSN import SubSpectralNorm

In [ ]:
def get_padding(kernel_size: Tuple[int, int],
                dilation: int):
    kh, kw = kernel_size
    ph = (kh-1)*dilation//2
    pw = (kw-1)*dilation//2
    return (ph, pw)

## BaseBlock

In [ ]:
class BaseBlock(nn.Module):
    """
    Base block

    Args:
    kernel_size: Tuple
        kernel_size for both freq_conv and temporal_conv

    """
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, int],
                 stride: Tuple[int, int],
                 dilation: Tuple[int, int],
                 bias: bool = True,
                 ssn_kwargs: dict = None,
                 dropout: float = 0.1
                 ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # Set up arguments
        f_kernel = (kernel_size[0], 1)
        t_kernel = (1, kernel_size[1])
        f_stride = (stride[0], 1)
        t_stride = (1, stride[1])
        f_padding = get_padding(f_kernel, dilation[0])
        t_padding = get_padding(t_kernel, dilation[1])

        # Freq_conv
        f2 = []
        if in_channels != out_channels:
            f2.extend([nn.Conv2d(in_channels,
                                out_channels,
                                kernel_size = 1,
                                bias = bias),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU(inplace = True)])


        self.f2 = nn.Sequential(*f2,
                                nn.Conv2d(out_channels,
                                          out_channels,
                                          kernel_size = f_kernel,
                                          stride = f_stride,
                                          padding = f_padding,
                                          dilation = dilation[0],
                                          groups = out_channels,
                                          bias = bias
                                          ),
                                SubSpectralNorm(out_channels, **ssn_kwargs)
                                if ssn_kwargs else nn.BatchNorm2d(out_channels))
        # Temporal_conv
        self.f1 = nn.Sequential(nn.AdaptiveAvgPool2d((1, None)),
                                nn.Conv2d(out_channels,
                                          out_channels,
                                          kernel_size = t_kernel,
                                          stride = t_stride,
                                          dilation = dilation[1],
                                          padding = t_padding,
                                          groups = out_channels,
                                          bias = bias),
                                nn.BatchNorm2d(out_channels),
                                nn.SiLU(inplace = True),
                                nn.Conv2d(out_channels,
                                          out_channels,
                                          kernel_size = 1,
                                          bias = bias),
                                nn.Dropout(dropout))
        self.relu = nn.ReLU(inplace = True)

    def forward(self,
                input: Tensor) -> Tensor:
        auxiliary_x = self.f2(input)
        output = self.f1(auxiliary_x)
        if self.in_channels == self.out_channels:
            output = output + auxiliary_x + input
        else:
            output = output + auxiliary_x
        return self.relu(output)

In [ ]:
input = torch.Tensor(128, 8, 20, 101)
bcblock = BaseBlock(8,
                    12,
                    kernel_size = (3, 3),
                    stride = (2, 1),
                    dilation = (1, 2),
                    bias = False,
                    ssn_kwargs = {
                        'spec_groups': 5
                    })
bcblock(input).shape

torch.Size([128, 12, 10, 101])

## BC-ResBlock

In [ ]:
class BCResBlock(nn.Module):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, int],
                 stride: Tuple[int, int],
                 bias: bool = True,
                 ssn_kwargs: dict = None,
                 dropout: float = 0.1,
                 num_blks: int = 1,
                 idx: int = 0
                 ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_blks = num_blks

        blks = []
        for i in range(num_blks):
            blks.append(BaseBlock(in_channels if i==0 else out_channels,
                                  out_channels,
                                  kernel_size,
                                  stride if i == 0 else (1, 1),
                                  dilation = (1, 2**idx),
                                  bias = False,
                                  ssn_kwargs = {
                                      'spec_groups': 5
                                  }))
        self.blks = nn.Sequential(*blks)

    def forward(self,
                input: Tensor) -> Tensor:
        return self.blks(input)

In [ ]:
input = torch.Tensor(128, 8, 20, 101)
bcblock = BCResBlock(8,
                    12,
                    kernel_size = (3, 3),
                    stride = (2, 1),
                    bias = False,
                    ssn_kwargs = {
                        'spec_groups': 5
                    })
bcblock(input).shape

torch.Size([128, 12, 10, 101])

## BC-ResNet

In [ ]:
class BCResNet(nn.Module):
    def __init__(self,
                 in_channels: int,
                 num_classes: int,
                 bias: bool = False,
                 mul_factor: int = 1) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.mul_factor = mul_factor
        self.net = nn.Sequential(nn.Conv2d(in_channels,
                                           int(16*mul_factor),
                                           kernel_size = 5,
                                           padding = 2,
                                           stride = (1, 1),
                                           dilation = 1,
                                           bias = bias),
                                 nn.BatchNorm2d(int(16*mul_factor)),
                                 nn.ReLU(inplace = True),
                                 BCResBlock(int(16*mul_factor),
                                            int(8*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = bias,
                                            ssn_kwargs = {
                                                'spec_groups': 5
                                            },
                                            num_blks = 2,
                                            idx = 0),
                                 BCResBlock(int(8*mul_factor),
                                            int(12*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = bias,
                                            ssn_kwargs = {
                                                'spec_groups': 5
                                            },
                                            num_blks = 2,
                                            idx = 1),
                                 BCResBlock(int(12*mul_factor),
                                            int(16*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = bias,
                                            ssn_kwargs = {
                                                'spec_groups': 5
                                            },
                                            num_blks = 4,
                                            idx = 2),
                                 BCResBlock(int(16*mul_factor),
                                            int(20*mul_factor),
                                            kernel_size = (3, 3),
                                            stride = (1, 1),
                                            bias = False,
                                            ssn_kwargs = {
                                                'spec_groups': 5
                                            },
                                            num_blks = 4,
                                            idx = 3),
                                 nn.Conv2d(int(20*mul_factor),
                                           int(20*mul_factor),
                                           kernel_size = 5,
                                           padding = (0, 2),
                                           bias  = bias,
                                           groups = int(20*mul_factor)),
                                 nn.Conv2d(int(20*mul_factor),
                                           int(32*mul_factor),
                                           kernel_size = 1,
                                           bias = bias),
                                 nn.BatchNorm2d(int(32*mul_factor)),
                                 nn.ReLU(inplace = True),
                                 nn.AdaptiveAvgPool2d((1, 1)),
                                 nn.Conv2d(int(32*mul_factor),
                                           num_classes,
                                           kernel_size = 1)
                                 )
    def forward(self, input:Tensor) -> Tensor:
        x = self.net(input)
        return x.reshape(-1, x.shape[1])

In [ ]:
input = torch.Tensor(1, 1, 40, 101)
model = BCResNet(1,
                   12,
                   False,
                    3)
model(input).shape

torch.Size([1, 12])

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

In [ ]:
summary(model, input_size = (1, 1, 40, 101))

Layer (type:depth-idx)                                  Output Shape              Param #
BCResNet                                                [1, 12]                   --
├─Sequential: 1-1                                       [1, 12, 1, 1]             --
│    └─Conv2d: 2-1                                      [1, 16, 40, 101]          400
│    └─BatchNorm2d: 2-2                                 [1, 16, 40, 101]          32
│    └─ReLU: 2-3                                        [1, 16, 40, 101]          --
│    └─BCResBlock: 2-4                                  [1, 8, 40, 101]           --
│    │    └─Sequential: 3-1                             [1, 8, 40, 101]           560
│    └─BCResBlock: 2-5                                  [1, 12, 40, 101]          --
│    │    └─Sequential: 3-2                             [1, 12, 40, 101]          840
│    └─BCResBlock: 2-6                                  [1, 16, 40, 101]          --
│    │    └─Sequential: 3-3                             [

In [ ]:
dilation = 3
kernel_size = (5, 3)
padding = get_padding(kernel_size, dilation)
a = torch.Tensor(128, 1, 40, 101)
conv2d = nn.Conv2d(1, 3,
                   kernel_size = kernel_size,
                   dilation = 3,
                   padding = padding)
conv2d(a).shape

torch.Size([128, 3, 40, 101])

In [ ]:
dilation = 3
kernel_size = 5
padding = dilation*(kernel_size - 1)//2
a = torch.Tensor(128, 40, 101)
conv1d = nn.Conv1d(40, 40,
                   kernel_size = 5,
                   dilation = 3,
                   padding = padding)
conv1d(a).shape

torch.Size([128, 40, 101])

In [ ]:
f_avg_pool = nn.AdaptiveAvgPool2d((1, None))
f_avg_pool(a).shape

torch.Size([128, 1, 1, 101])